In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset, load_metric
from datasets import Sequence, ClassLabel
from datasets import Value

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [5]:
# dataset = load_dataset("conll2003", revision="master")

In [6]:
# dataset

In [7]:
PE_DATA_FILE = '/notebooks/token_classification/data/pe_dataset_task_1_w_ints.pt'

In [8]:
pe_dataset = torch.load(PE_DATA_FILE)

In [9]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int'],
        num_rows: 273
    })
})

In [10]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [11]:
pe_dataset = pe_dataset.rename_column("tags_list_int", "paragraph_tags_list_int")

In [12]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

### my work today

In [13]:
# what if we sequence the int labels to the class labels? will it work?

In [14]:
pe_dataset['train'].features['paragraph_tags_list_int'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [15]:
pe_dataset['train'].features['paragraph_tags_list_int']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [16]:
pe_dataset['test'].features['paragraph_tags_list_int'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [17]:
pe_dataset['validation'].features['paragraph_tags_list_int'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [18]:
pe_dataset['test'].features['paragraph_tags_list_int']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [19]:
pe_dataset['validation'].features['paragraph_tags_list_int']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [20]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_list_int': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)}

In [21]:
pe_dataset["train"][0]["paragraph_tags_list_int"]

[0,
 0,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 0]

In [22]:
pe_dataset["train"].features["paragraph_tags_list_int"]

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [23]:
# seems correct

In [24]:
from transformers import AutoTokenizer

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [26]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [27]:
def tokenize_and_align_labels_hf(examples):
    tokenized_inputs = tokenizer(
        examples["paragraph_words"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["paragraph_tags_list_int"]
    #print(examples["paragraph_tags"])
    #all_labels = labels_object.str2int(examples["paragraph_tags"])
    
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [28]:
label_all_tokens = True

In [29]:
tokenized_datasets = pe_dataset.map(tokenize_and_align_labels_hf, batched=True)

Parameter 'function'=<function tokenize_and_align_labels_hf at 0x7efe8b401310> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
label_list = pe_dataset["train"].features['paragraph_tags_list_int'].feature.names

In [31]:
label_list

['O',
 'B-MajorClaim',
 'I-MajorClaim',
 'B-Claim',
 'I-Claim',
 'B-Premise',
 'I-Premise']

In [32]:
pe_dataset["train"].features["paragraph_tags_list_int"].feature.names

['O',
 'B-MajorClaim',
 'I-MajorClaim',
 'B-Claim',
 'I-Claim',
 'B-Premise',
 'I-Premise']

In [33]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [35]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=32,
    weight_decay=0.01,
    eval_steps=20,
    logging_steps=20
    #push_to_hub=True,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running training *****
  Num examples = 1091
  Num Epochs = 32
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2208


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.016100,0.923098,0.173538,0.309550,0.222397,0.661934
2,0.808200,0.743271,0.169419,0.349067,0.228121,0.723960
3,0.682300,0.676964,0.189112,0.362239,0.248494,0.751803
4,0.585900,0.651583,0.217702,0.434687,0.290110,0.774041
5,0.467600,0.623449,0.270781,0.472009,0.344138,0.782357
6,0.405400,0.610535,0.277281,0.510428,0.359351,0.785711
7,0.325700,0.638756,0.332237,0.554336,0.415467,0.791684
8,0.282200,0.676299,0.351279,0.572997,0.435544,0.794946
9,0.235500,0.705739,0.341695,0.553238,0.422464,0.778911
10,0.219500,0.745644,0.385551,0.591658,0.466869,0.789203


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16
The following columns in the evaluation set  don't h

TrainOutput(global_step=2208, training_loss=0.20678682924261776, metrics={'train_runtime': 608.2972, 'train_samples_per_second': 57.393, 'train_steps_per_second': 3.63, 'total_flos': 2595680085325008.0, 'train_loss': 0.20678682924261776, 'epoch': 32.0})

In [41]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16


{'eval_loss': 1.146390438079834,
 'eval_precision': 0.44332298136645965,
 'eval_recall': 0.6267837541163557,
 'eval_f1': 0.5193269668030923,
 'eval_accuracy': 0.7966459912703883,
 'eval_runtime': 1.8549,
 'eval_samples_per_second': 147.176,
 'eval_steps_per_second': 9.704,
 'epoch': 32.0}

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Prediction *****
  Num examples = 355
  Batch size = 16


{'Claim': {'precision': 0.3268101761252446,
  'recall': 0.5548172757475083,
  'f1': 0.4113300492610837,
  'number': 301},
 'MajorClaim': {'precision': 0.40425531914893614,
  'recall': 0.6291390728476821,
  'f1': 0.4922279792746114,
  'number': 151},
 'Premise': {'precision': 0.5932028836251287,
  'recall': 0.715527950310559,
  'f1': 0.6486486486486487,
  'number': 805},
 'overall_precision': 0.48806057076295867,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.5635507733691998,
 'overall_accuracy': 0.8284271032921093}

In [43]:
# task: try to do only with the BIO. then see the results.

In [44]:
predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results_train = metric.compute(predictions=true_predictions, references=true_labels)
results_train

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: split, essay_nr, para_nr, paragraph, essay, paragraph_tags, paragraph_words, paragraph_tags_list_int.
***** Running Prediction *****
  Num examples = 1091
  Batch size = 16


{'Claim': {'precision': 0.9759414225941423,
  'recall': 0.9821052631578947,
  'f1': 0.9790136411332634,
  'number': 950},
 'MajorClaim': {'precision': 0.9586956521739131,
  'recall': 0.9735099337748344,
  'f1': 0.9660460021905805,
  'number': 453},
 'Premise': {'precision': 0.9670646356525319,
  'recall': 0.9828451882845188,
  'f1': 0.9748910562357336,
  'number': 2390},
 'overall_precision': 0.9682704811443433,
 'overall_recall': 0.9815449512259425,
 'overall_f1': 0.9748625294579734,
 'overall_accuracy': 0.9983153788181053}